<a href="https://colab.research.google.com/github/frstndrd/Monografia/blob/main/tcc_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [194]:
!pip install mip

In [195]:
import pandas as pd
from datetime import timedelta

# Funcao para converter o formato de hora, considerando horarios acima de 24 horas
def convert_time(t):
    if isinstance(t, str):  # Verifica se t eh uma string
        hours, minutes = map(int, t.split(':'))
        if hours >= 24:
            hours -= 24
            return timedelta(days=1, hours=hours, minutes=minutes)
        return timedelta(hours=hours, minutes=minutes)
    else:
        return timedelta(0)

# Carregar as tarefas de um arquivo .trf hospedado no GitHub
url = 'https://raw.githubusercontent.com/frstndrd/Monografia/refs/heads/main/entrada/matriz_transcom_do_s2.trf' # 92
# url = 'https://raw.githubusercontent.com/frstndrd/Monografia/refs/heads/main/entrada/matriz_transcom_sa_s2.trf' # 121
# url = 'https://raw.githubusercontent.com/frstndrd/Monografia/refs/heads/main/entrada/matriz_transcom_du_s2.trf' # 153
# url = 'https://raw.githubusercontent.com/frstndrd/Monografia/refs/heads/main/entrada/matriz_setop_do.trf' # 169
# url = 'https://raw.githubusercontent.com/frstndrd/Monografia/refs/heads/main/entrada/matriz_setop_sa.trf' # 262
# url = 'https://raw.githubusercontent.com/frstndrd/Monografia/refs/heads/main/entrada/matriz_setop_du.trf' # 384
# url = 'https://raw.githubusercontent.com/frstndrd/Monografia/refs/heads/main/teste.trf'


tasks = pd.read_csv(url, sep='\s+', skiprows=2)

# Converter horario de inicio e fim para timedeltas
tasks['ST'] = tasks['ST'].apply(convert_time)
tasks['ET'] = tasks['ET'].apply(convert_time)

print(tasks.head())

# Parametros
min_task_duration = timedelta(minutes=60)
max_gap_between_tasks = timedelta(minutes=120)
min_work_time = timedelta(minutes=60)
min_short_gap = timedelta(minutes=15)
min_long_gap = timedelta(minutes=30)

max_overtime = timedelta(hours=2)
max_shift_duration = timedelta(hours=6, minutes=40)
shift_generation_limit = 100

idle_penalty = 0.05

# Funcao de geracao de Shift-Pool
def generate_shift_pool(tasks):
    nW = len(tasks)
    shift_pool = []

    # Adicionar cada tarefa como um jornada de uma unica tarefa
    for i in range(nW):
        task_i = tasks.iloc[i]
        shift_pool.append((task_i['ID'],))  # Single-task shift

    # Agora adicionar apenas jornadas de duas tarefas
    for i in range(nW - 1):
        task_i = tasks.iloc[i]
        start_i = task_i['ST']
        end_i = task_i['ET']

        # Garantir que a tarefa i atenda a duracao minima da tarefa
        if end_i - start_i >= min_task_duration:
            for j in range(i + 1, nW):
                task_j = tasks.iloc[j]
                start_j = task_j['ST']
                end_j = task_j['ET']

                # Garantir que a tarefa j atenda a duracao minima da tarefa
                if end_j - start_j >= min_task_duration:
                    idle_time = start_j - end_i
                    total_work_time = (end_i - start_i) + (end_j - start_j)

                    if timedelta(minutes=0) < idle_time <= max_gap_between_tasks:
                        if total_work_time <= max_shift_duration:
                            shift_pool.append((task_i['ID'], task_j['ID']))
    return shift_pool

# Validacao de jornadas (EL = SL, sem overlap de horário)
def valid_shift(task_i, task_j, idle_time, total_work_time):
    loc_i_end = task_i['EL']
    loc_j_start = task_j['SL']

    # Verificacao basica: tarefa_j deve comecar apos tarefa_i terminar
    if idle_time.total_seconds() > 0:
        return True

    return False

# Gerar o shift-pool
shift_pool = generate_shift_pool(tasks)
print("Shift Pool:", shift_pool)
print(f"Shift Pool Size: {len(shift_pool)}")

   ID              ST  SL              ET  EL  OC  VC Linha
0   1 0 days 04:45:00   0 0 days 06:50:00  13   0  28  302D
1   2 0 days 04:50:00   0 0 days 07:00:00  14   0  29  302A
2   3 0 days 05:15:00   0 0 days 07:10:00  11   0  30  302B
3   4 0 days 05:15:00   0 0 days 07:30:00  12   0  31  302C
4   5 0 days 05:20:00   0 0 days 07:40:00  14   0  32   370
Shift Pool: [(1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (10,), (11,), (12,), (13,), (14,), (15,), (16,), (17,), (18,), (19,), (20,), (21,), (22,), (23,), (24,), (25,), (26,), (27,), (28,), (29,), (30,), (31,), (32,), (33,), (34,), (35,), (36,), (37,), (38,), (39,), (40,), (41,), (42,), (43,), (44,), (45,), (46,), (47,), (48,), (49,), (50,), (51,), (52,), (53,), (54,), (55,), (56,), (57,), (58,), (59,), (60,), (61,), (62,), (63,), (64,), (65,), (66,), (67,), (68,), (69,), (70,), (71,), (72,), (73,), (74,), (75,), (76,), (77,), (78,), (79,), (80,), (81,), (82,), (83,), (84,), (85,), (86,), (87,), (88,), (89,), (90,), (91,),

In [196]:
print(tasks)

    ID              ST  SL              ET  EL  OC  VC Linha
0    1 0 days 04:45:00   0 0 days 06:50:00  13   0  28  302D
1    2 0 days 04:50:00   0 0 days 07:00:00  14   0  29  302A
2    3 0 days 05:15:00   0 0 days 07:10:00  11   0  30  302B
3    4 0 days 05:15:00   0 0 days 07:30:00  12   0  31  302C
4    5 0 days 05:20:00   0 0 days 07:40:00  14   0  32   370
..  ..             ...  ..             ...  ..  ..  ..   ...
87  88 0 days 21:50:00  13 1 days 00:10:00   0   0  28  302D
88  89 0 days 22:00:00  14 1 days 00:40:00   0   0  35  302A
89  90 0 days 22:30:00  11 1 days 00:50:00   0   0  36  302B
90  91 0 days 22:40:00  14 1 days 01:15:00   0   0  37   370
91  92 0 days 23:00:00  13 1 days 01:30:00   0   0  34  302D

[92 rows x 8 columns]


In [197]:
from mip import Model, xsum, minimize, OptimizationStatus, CONTINUOUS
import time
from datetime import timedelta

# Funcao para minimizar jornadas, incentivar dupla pegada e horas extras, e penalizar tempo ocioso
def restricted_master_problem(shift_pool, n_tasks, silent=True, rel_gap=1e-6,
                              overtime_reward=100.0, split_shift_reward=1.0,
                              idle_time_penalty=0.5, shift_penalty=10000.0):
    rmp = Model()

    # Variaveis de decisao (continuas entre 0 e 1 para relaxamento)
    shift_vars = [rmp.add_var(var_type=CONTINUOUS, lb=0, ub=1) for _ in range(len(shift_pool))]

    total_overtime = []
    total_idle_time = []
    split_shift_rewards = []

    for i, shift in enumerate(shift_pool):
        # Calcular tempo de trabalho, tempo ocioso, horas extras, dupla pegada e validade da jornada
        work_time, idle_time, overtime, is_split_shift, is_valid_shift = calculate_shift_metrics(
            shift, tasks, max_shift_duration, max_overtime,
            min_short_gap=timedelta(minutes=15), min_long_gap=timedelta(minutes=30)
        )

        # Ja que eh o problema mestre restrito, assumir que todas as jornadas geradas sao validas
        total_overtime.append(overtime)
        total_idle_time.append(idle_time)
        split_shift_rewards.append(is_split_shift)  # 1 se for dupla pegada, 0 caso contrario

    # Objetivo: Minimizar o número de jornadas, incentivar horas extras e dupla pegada, e penalizar tempo ocioso
    rmp.objective = minimize(
        shift_penalty * xsum(shift_vars)
        - overtime_reward * xsum(shift_vars[i] * total_overtime[i] for i in range(len(shift_vars)))
        - split_shift_reward * xsum(shift_vars[i] * split_shift_rewards[i] for i in range(len(shift_vars)))
        + idle_time_penalty * xsum(shift_vars[i] * total_idle_time[i] for i in range(len(shift_vars)))
    )

    # Restricoes: Cada tarefa deve ser coberta pelo menos uma vez
    task_constraints = {}
    for task in range(1, n_tasks + 1):
        task_constraints[task] = rmp.add_constr(
            xsum(shift_vars[i] for i, shift in enumerate(shift_pool) if task in shift) == 1
        )

    # Resolver o RMP com uma folga relativa quase zero (para otimizacao exata)
    rmp.rel_gap = rel_gap
    status = rmp.optimize()

    if not silent:
        if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
            print(f"Valor final da funcao objetivo: {rmp.objective_value}")
        else:
            print(f"Aviso: Nenhuma solucao valida encontrada (status: {status}).")

    # Collect dual variables and selected shifts for column generation
    if status in {OptimizationStatus.OPTIMAL, OptimizationStatus.FEASIBLE}:
        dual_vars = [task_constraints[task].pi if task_constraints[task].pi is not None else 0
                     for task in range(1, n_tasks + 1)]
        selected_shifts = [shift_pool[i] for i in range(len(shift_pool)) if shift_vars[i].x >= 0.5]
    else:
        return None, [], []

    return rmp, dual_vars, selected_shifts


def calculate_shift_metrics(shift, tasks, max_shift_duration, max_overtime, min_short_gap=timedelta(minutes=15), min_long_gap=timedelta(minutes=30)):
    total_work_time = timedelta(0)
    idle_time = timedelta(0)
    overtime = timedelta(0)
    is_split_shift = 0

    last_end_time = None

    for task_id in shift:
        task = tasks[tasks['ID'] == task_id].iloc[0]
        start = task['ST']
        end = task['ET']

        work_time = end - start
        total_work_time += work_time

        if last_end_time is not None:
            idle_period = start - last_end_time
            idle_time += idle_period

            # Se o tempo ocioso for maior que 2 horas, eh uma dupla pegada
            if idle_period > timedelta(hours=2):
                is_split_shift = 1

        last_end_time = end

    # Calcular horas extras se o tempo total de trabalho exceder a duracao maxima da jornada
    total_paid_time = total_work_time + idle_time
    if total_paid_time > max_shift_duration:
        overtime = total_paid_time - max_shift_duration
        overtime = min(overtime, max_overtime)  # Cap overtime at max allowed overtime

    # Garantir que o tempo total da jornada nao exceda 8h40m
    total_shift_time = total_work_time + idle_time
    is_valid_shift = total_shift_time <= (max_shift_duration + max_overtime)

    # Determinar se a jornada eh "curta" ou "longa" para os requisitos de intervalo
    if timedelta(hours=4) <= total_work_time <= timedelta(hours=6):
        allowed_break = min_short_gap
    elif total_work_time > timedelta(hours=6):
        allowed_break = min_long_gap
    else:
        allowed_break = timedelta(0)

    # Ajustar tempo ocioso subtraindo o tempo de intervalo permitido
    idle_time_after_break = max(idle_time - allowed_break, timedelta(0))

    # Retornar os tempos em horas (valores numéricos), o sinalizador de dupla pegada e a validade da jornada
    return total_work_time.total_seconds() / 3600, idle_time_after_break.total_seconds() / 3600, overtime.total_seconds() / 3600, is_split_shift, is_valid_shift



# Processo de geracao de colunas com depuracao aprimorada e calculo manual da folga de otimalidade
def column_generation(tasks, initial_shift_pool, rel_gap=1e-6):
    n_tasks = tasks['ID'].max()

    shift_pool = initial_shift_pool.copy()
    all_shifts = set(tuple(shift) for shift in shift_pool)
    final_shifts = []

    upper_bound = None
    lower_bound = None  # Sera definido com base na primeira solucao viavel
    iteration = 0

    while True:
        iteration += 1
        print(f"\nIteracao {iteration}: Resolvendo o Problema Mestre Restrito...")

        # Medir o tempo gasto para resolver o PMR
        start_rmp_time = time.time()

        # Resolver o PMR para a iteracao atual
        rmp, dual_vars, selected_shifts = restricted_master_problem(
            shift_pool, n_tasks, silent=True, rel_gap=rel_gap, idle_time_penalty=5.0)

        end_rmp_time = time.time()
        rmp_time_taken = end_rmp_time - start_rmp_time
        print(f"Iteracao {iteration}: RMP resolvido em {rmp_time_taken:.2f} segundos.")

        # Se nenhuma solucao valida for encontrada, sair
        if rmp is None:
            print(f"Nenhuma solucao valida, pulando esta iteracao.")
            break

        # Capturar o limite superior (valor da funcao objetivo)
        upper_bound = rmp.objective_value

        # Se esta for a primeira iteracao, usar a solucao atual como estimativa do limite inferior
        if lower_bound is None:
            lower_bound = upper_bound

        # Se a folga de otimalidade for pequena o suficiente, parar mais cedo
        if rmp.gap is not None and rmp.gap < rel_gap:
            print(f"Valor da funcao objetivo = {upper_bound}")
            print(f"Folga de otimalidade esta abaixo de {rel_gap * 100:.2f}%. Interrompendo.")
            final_shifts = selected_shifts
            break

        # Medir o tempo gasto para gerar novas jornadas
        start_pricing_time = time.time()

        # Resolver o problema de precificacao para gerar novas jornadas
        new_columns = pricing_problem(tasks, dual_vars, shift_pool)

        end_pricing_time = time.time()
        pricing_time_taken = end_pricing_time - start_pricing_time
        print(f"Iteracao {iteration}: Problema de pricing resolvido em {pricing_time_taken:.2f} segundos.")

        # Adicionar novas colunas unicas ao pool de jornadas
        new_unique_columns = [tuple(shift) for shift in new_columns if tuple(shift) not in all_shifts]
        shift_pool += new_unique_columns
        all_shifts.update(new_unique_columns)

        # Parar se nenhuma nova coluna for gerada
        if not new_unique_columns:
            print(f"Iteracao {iteration}: Nenhuma nova jornada gerada. Interrompendo.")
            final_shifts = selected_shifts
            break

    # Resolver o RMP final para relatar as metricas finais
    print("\nResolucao Final do Problema Mestre Restrito...")
    rmp, dual_vars, final_shifts = restricted_master_problem(
        shift_pool, n_tasks, silent=False, rel_gap=rel_gap)

    # Imprimir a tabela de detalhes das jornadas
    print_shift_details(final_shifts, tasks)

    return final_shifts


# Problema de precificacao para gerar jornadas com múltiplas tarefas, incluindo a combinacao de jornadas existentes
def pricing_problem(tasks, dual_vars, shift_pool):
    new_columns = []
    nW = len(tasks)

    # Debug: Acompanhar quantas jornadas sao verificadas e quantas se tornam colunas validas
    total_checked = 0
    total_valid = 0

    # Jornadas de uma unica tarefa
    for i in range(nW):
        task_i = tasks.iloc[i]
        new_shift = (task_i['ID'],)
        reduced_cost = calculate_reduced_cost(new_shift, dual_vars)

        total_checked += 1  # Count how many shifts are checked

        # Validar se a jornada esta dentro do tempo permitido usando calculate_shift_metrics
        work_time, idle_time, overtime, _, is_valid_shift = calculate_shift_metrics(new_shift, tasks, max_shift_duration, max_overtime)

        # Pular jornadas "ruins" que excedam o tempo total de 8h40m
        if reduced_cost < 0 and is_valid_shift:
            new_columns.append(new_shift)
            total_valid += 1

    # Jornadas de duas tarefas
    for i in range(nW - 1):
        task_i = tasks.iloc[i]
        end_i = task_i['ET']

        for j in range(i + 1, nW):
            task_j = tasks.iloc[j]
            start_j = task_j['ST']
            end_j = task_j['ET']

            idle_time = start_j - end_i
            total_work_time = (end_i - task_i['ST']) + (end_j - start_j)

            if valid_shift(task_i, task_j, idle_time, total_work_time):
                new_shift = (task_i['ID'], task_j['ID'])
                reduced_cost = calculate_reduced_cost(new_shift, dual_vars)

                total_checked += 1

                # Validar se a jornada esta dentro do tempo permitido
                work_time, idle_time, overtime, _, is_valid_shift = calculate_shift_metrics(new_shift, tasks, max_shift_duration, max_overtime)

                # Pular jornadas "ruins" que excedam o tempo total de 8h40m
                if reduced_cost < 0 and is_valid_shift:
                    new_columns.append(new_shift)
                    total_valid += 1

    # Combinar jornadas existentes com multiplas tarefas com tarefas adicionais
    for shift in shift_pool:
        if len(shift) == 2:  # Considerar apenas jornadas de duas tarefas para extensao
            task_i_id, task_j_id = shift

            task_i = tasks[tasks['ID'] == task_i_id].iloc[0]
            task_j = tasks[tasks['ID'] == task_j_id].iloc[0]
            end_j = task_j['ET']
            loc_j = task_j['EL']

            for k in range(nW):
                task_k = tasks.iloc[k]
                start_k = task_k['ST']
                end_k = task_k['ET']
                loc_k = task_k['SL']

                idle_time = start_k - end_j
                total_work_time = (end_j - task_i['ST']) + (end_k - start_k)

                if valid_shift(task_j, task_k, idle_time, total_work_time):
                    extended_shift = (*shift, task_k['ID'])
                    reduced_cost = calculate_reduced_cost(extended_shift, dual_vars)

                    total_checked += 1

                    # Validar se a jornada esta dentro do tempo permitido
                    work_time, idle_time, overtime, _, is_valid_shift = calculate_shift_metrics(extended_shift, tasks, max_shift_duration, max_overtime)

                    # Pular jornadas "ruins" que excedam o tempo total de 8h40m
                    if reduced_cost < 0 and is_valid_shift:
                        new_columns.append(extended_shift)
                        total_valid += 1  # Count valid shifts added to columns

    # Debug: Imprimir quantas jornadas foram verificadas e quantas foram colunas validas
    print(f"Pricing Problem: {total_checked} shifts checked, {total_valid} valid columns generated.")

    return new_columns


# Funcao para calcular o custo reduzido com base nas variaveis duais
def calculate_reduced_cost(shift, dual_vars):
    reduced_cost = sum(dual_vars[task - 1] for task in shift) - 1
    return reduced_cost


# Funcao para formatar valores de timedelta para impressao
def format_timedelta(td):
    days = td.days
    hours, remainder = divmod(td.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    if days == 0:
        return f"0 days {hours:02}:{minutes:02}:{seconds:02}"
    else:
        return f"{days} days {hours:02}:{minutes:02}:{seconds:02}"


# Funcao atualizada para imprimir os detalhes das jornadas e acompanhar jornadas com dupla pegada e total de horas extras
def print_shift_details(final_shifts, tasks):
    assigned_tasks = set()  # Garantir que cada tarefa seja alocada apenas uma vez
    split_shift_count = 0   # Contar o numero de jornadas com dupla pegada
    total_overtime = timedelta(0)  # Acompanhar o total de horas extras

    print(f"\n{'Shift':<10}\t{'Work Time':<25}{'Idle Time':<25}{'Overtime':<25}{'Split Shift?':<15}")
    print("=" * 100)

    for shift in final_shifts:
        if not shift or any(task_id in assigned_tasks for task_id in shift):  # Ignorar jornadas vazias ou ja alocadas
            continue

        shift_tasks = []
        total_work_time = timedelta(0)
        last_end_time = None
        is_split_shift = False

        # Processar cada tarefa na jornada
        for task_id in shift:
            if task_id in assigned_tasks:
                continue  # Ignorar se a tarefa ja foi alocada a outra jornada

            task = tasks[tasks['ID'] == task_id].iloc[0]
            start = task['ST']
            end = task['ET']

            shift_tasks.append(task_id)
            work_time = end - start
            total_work_time += work_time

            assigned_tasks.add(task_id)  # Marcar a tarefa como alocada

            if last_end_time is not None:
                idle_time = start - last_end_time
                if idle_time > timedelta(hours=2):
                    is_split_shift = True  # Marcar como uma jornada com dupla pegada

            last_end_time = end

        # Se nenhuma tarefa foi alocada nessa jornada, ignorar
        if not shift_tasks:
            continue

        # Calcular tempo ocioso: qualquer tempo nao trabalhado dentro da duracao maxima da jornada
        total_paid_time = max_shift_duration  # Limite de jornada de 6h40m
        idle_time = max(timedelta(0), total_paid_time - total_work_time)

        # Calcular horas extras
        overtime = max(timedelta(0), total_work_time - max_shift_duration)
        overtime = min(overtime, max_overtime)  # Cap overtime

        # Adicionar ao total de horas extras
        total_overtime += overtime

        # Contar jornadas com dupla pegada
        if is_split_shift:
            split_shift_count += 1

        # Imprimir detalhes
        print(f"{'-'.join(map(str, shift_tasks))}\t\t{format_timedelta(total_work_time)}\t\t"
              f"{format_timedelta(idle_time)}\t\t{format_timedelta(overtime)}\t\t{'Yes' if is_split_shift else 'No'}")

    # Imprimir o número de jornadas com dupla pegada e o total de horas extras
    print("\nSummary:")
    print(f"Total Split Shifts: {split_shift_count}")
    print(f"Total Overtime: {format_timedelta(total_overtime)}")


start_time = time.time()
final_shifts = column_generation(tasks, shift_pool, rel_gap=1e-6)
total_time = time.time() - start_time
print(f"Column generation completed in {total_time:.2f} seconds.")


Iteration 1: Solving Restricted Master Problem...
Iteration 1: RMP solved in 5.35 seconds.
Pricing Problem: 40218 shifts checked, 0 valid columns generated.
Iteration 1: Pricing problem solved in 116.82 seconds.
Iteration 1: No new shifts generated. Stopping.

Final Solve of Restricted Master Problem...
Final Objective function value: 453992.62500000006

Shift     	Work Time                Idle Time                Overtime                 Split Shift?   
1-32		0 days 04:40:00		0 days 02:00:00		0 days 00:00:00		Yes
2-31		0 days 04:00:00		0 days 02:40:00		0 days 00:00:00		Yes
3-34		0 days 04:25:00		0 days 02:15:00		0 days 00:00:00		Yes
4-37		0 days 04:25:00		0 days 02:15:00		0 days 00:00:00		Yes
5-33		0 days 04:20:00		0 days 02:20:00		0 days 00:00:00		Yes
6-17		0 days 04:20:00		0 days 02:20:00		0 days 00:00:00		No
7-19		0 days 03:55:00		0 days 02:45:00		0 days 00:00:00		No
8-38		0 days 04:40:00		0 days 02:00:00		0 days 00:00:00		Yes
9-20		0 days 03:55:00		0 days 02:45:00		0 days 00:00:0

In [198]:
# !cat /proc/cpuinfo